In [1]:
import os

import numpy as np

import figure_setup as fs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.collections import LineCollection
from matplotlib.patches import Arc

import ssptools
import astropy.units as u
from sbpy.photometry import HG1G2
from astropy.coordinates import SkyCoord
from scipy.signal import argrelextrema

In [2]:
# Choice target
sso_number = 22

# Spin
ra0 = np.radians(195)
dec0 = np.radians(-3)
R = 0.8

# Phase
H = 6.6
G1 = 0.29
G2 = 0.34

In [3]:
# Get ephemerides
nbd, step = 200, '10d'
ephemcc = ssptools.ephemcc(22, '2023-01-01', nbd=nbd, step=step, observer='010' )
ephxyz = ssptools.ephemcc(22, '2023-01-01', nbd=nbd, step=step, tcoor=2, rplane='1', observer='010' )

In [4]:
jd_ref = 2460000

In [5]:
# Dist function
f = 5*np.log10( ephemcc.Dobs * ephemcc.Dhelio ).values

In [6]:
# Phase function
ph = np.radians( ephemcc.Phase )

arg = G1 * HG1G2._phi1(ph) + G2 * HG1G2._phi2(ph) + (1 - G1 - G2) * HG1G2._phi3(ph)
g = -2.5 * np.log10(arg)

In [7]:
# Spin function
coords = SkyCoord( ephemcc.RA, ephemcc.DEC, unit=(u.hourangle, u.deg) )
ra = coords.ra.radian
dec = coords.dec.radian

cosL = np.sin(dec)*np.sin(dec0) + np.cos(dec)*np.cos(dec0)*np.cos(ra-ra0)

s = 2.5*np.log10( 1 - (1-R)*np.abs(cosL) )

## Figure magnitude (f,g,s)vs time

In [8]:
fig, ax = plt.subplots(figsize=fs.figsize(0.5, aspect=0.75),
                       gridspec_kw={
                           'bottom': 0.08, 
                           'top': 0.98, 
                           'left': 0.08, 
                           'right': 0.96
                       }
                      )


# Find oppositions
p_oppo = ephemcc['Elong.'].isin( ephemcc['Elong.'].values[argrelextrema(ephemcc['Elong.'].values, np.greater)[0]] )
p_conj = ephemcc['Elong.'].isin( ephemcc['Elong.'].values[argrelextrema(ephemcc['Elong.'].values, np.less)[0]] )


# Date
date = ephemcc.Date - jd_ref

# Tune
xTxt = 2000
xBar = -200
iTxt = -1
sSize = 10
offsets = {'f': -2.0,
           'g': 0.01,
           's': 0.01,
           'm': 0.01 }
colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']


# Distance
yf = f+offsets['f']
ax.plot( date, yf, color=colors[0] )
ax.text( xTxt, yf[iTxt], 'f(r,$\\Delta$)' )
ax.errorbar( xBar, np.min(yf), yerr=np.array([[0, np.max(yf)-np.min(yf)]]).T, capsize=2, color=colors[0] )
ax.text( xBar-150, (np.max(yf)+np.min(yf))/2., f'{np.max(yf)-np.min(yf):4.2f}', rotation=90, va='center' )

# Phase
yg = g+offsets['g']
# ax.scatter( date, yg, s=sSize, rasterized=True )
ax.plot( date, yg, color=colors[1] )
ax.text( xTxt, yg[iTxt], 'g($\\gamma$)' )
ax.errorbar( xBar, np.min(yg), yerr=np.array([[0, np.max(yg)-np.min(yg)]]).T, capsize=2, color=colors[1] )
ax.text( xBar-150, (np.max(yg)+np.min(yg))/2., f'{np.max(yg)-np.min(yg):4.2f}', rotation=90, va='center' )

# Spin
ys = s+offsets['s']
ax.plot( date, ys, color=colors[3] )
ax.text( xTxt, ys[iTxt], 's($\\alpha,\\delta$)' )
ax.errorbar( xBar, np.min(ys), yerr=np.array([[0, np.max(ys)-np.min(ys)]]).T, capsize=2, color=colors[3] )
ax.text( xBar-150, (np.max(ys)+np.min(ys))/2., f'{np.max(ys)-np.min(ys):4.2f}', rotation=90, va='center' )


# Apparent magnitude
yH = f+g + offsets['m']
ysH = f+g+s + offsets['m']

ax.plot( date, yH, color='grey' )
ax.text( xTxt, yH[iTxt], 'HG$_1$G$_2$', color='grey' )

ax.plot( date, ysH, color='black' )
ax.text( xTxt, ysH[iTxt], 'sHG$_1$G$_2$' )

# Mark Opposition
for i,row in ephemcc[p_oppo].iterrows():
    jd_oppo = row['Date'] - jd_ref
    ax.annotate("", xy=(jd_oppo, yf[i]-0.025), xytext=(jd_oppo, yf[i]-0.4),
                arrowprops=dict(arrowstyle="->"))
ax.text( jd_oppo+50, yf[i]-0.4, 'Oppositions', va='bottom', fontsize='small')

for i,row in ephemcc[p_conj].iterrows():
    jd_conj = row['Date'] - jd_ref
    ax.annotate("", xy=(jd_conj, yf[i]+0.025), xytext=(jd_conj, yf[i]+0.4),
                arrowprops=dict(arrowstyle="->",color='grey'))
ax.text( jd_conj+50, yf[i]+0.4, 'Conjonctions', va='top', fontsize='small', color='grey')

# Axes
ax.set_xlabel(f'Days from {jd_ref:,d}')
ax.set_ylabel('Magnitude')

ax.set_xlim(-400,2500)
ax.set_ylim(-0.5,6.5)

fig.savefig( os.path.join('..','gfx','article','fig_model.pgf'))
fig.savefig( os.path.join('..','gfx','article','fig_model.png'), facecolor='white')
plt.close()

In [9]:
ephemcc[p_conj]

,Date,LAST,RA,DEC,HA,Az,H,Dobs,Dhelio,VMag,Phase,Elong.,AM,dRAcosDEC,dDEC,RV
32,2460265.5,04:10:52.25,+15:22:58.4785,-15:53:41.218,12.798269,23.782541,-60.343474,4.177853,3.190702,12.591855,0.934619,3.016677,-999.0,0.860948,-0.296264,-0.724442
75,2460695.5,08:26:11.60,+20:28:03.8040,-28:07:58.030,11.968833,358.468940,-74.377331,3.998788,3.031383,12.541582,2.964758,9.168134,-999.0,0.954256,0.197090,-0.310402
122,2461165.5,15:19:12.91,+02:52:22.3457,+12:39:03.041,12.447380,7.837622,-33.266686,3.641869,2.636342,11.922608,1.409834,3.687968,-999.0,1.027311,0.411969,0.005125
170,2461645.5,22:51:39.99,+10:39:59.2972,+19:23:07.857,12.194638,3.085813,-26.806751,3.870725,2.881693,12.396569,3.542702,10.150935,-999.0,0.939511,-0.375956,1.397882


In [10]:
ephemcc[p_oppo]

,Date,LAST,RA,DEC,HA,Az,H,Dobs,Dhelio,VMag,Phase,Elong.,AM,dRAcosDEC,dDEC,RV
9,2460035.5,13:04:04.40,+13:19:16.0374,+09:04:13.246,23.746766,173.424167,55.161924,2.102404,3.073009,11.309139,5.333715,163.385002,1.217737,-0.529490,0.138734,-1.019871
53,2460475.5,17:58:49.14,+17:44:49.3904,-30:15:17.119,0.233263,183.142608,15.924989,2.163548,3.172916,11.274310,2.471824,172.257011,3.599376,-0.540474,-0.130258,-2.172424
99,2460935.5,00:12:25.07,+23:59:14.8891,-22:22:50.904,0.219495,183.327403,23.799193,1.836105,2.799252,10.905726,7.278259,159.342010,2.464826,-0.517438,-0.155527,-0.035977
146,2461405.5,07:05:26.50,+06:59:12.1417,+35:22:41.142,0.103989,188.678576,81.541710,1.704079,2.671407,10.518748,4.755552,166.984652,1.010972,-0.560825,0.245459,0.025780
192,2461865.5,13:19:02.22,+13:27:45.8597,+07:26:18.446,23.854545,176.349418,53.637304,2.110825,3.085464,11.311448,5.039847,164.275299,1.241084,-0.536019,0.125242,-0.634991


## Figure magnitude vs phase

In [11]:
# Ephemerides
nbd, step = 400, '1d'
eph300 = ssptools.ephemcc(22, jd_ref+50, nbd=nbd, step=step, observer='010' )
xyz300 = ssptools.ephemcc(22, jd_ref+50, nbd=nbd, step=step, tcoor=2, rplane='1', observer='010' )

# Phase function
ph300 = np.radians( eph300.Phase )

arg = G1 * HG1G2._phi1(ph300) + G2 * HG1G2._phi2(ph300) + (1 - G1 - G2) * HG1G2._phi3(ph300)
g300 = -2.5 * np.log10(arg)


# Spin function
coords = SkyCoord( eph300.RA, eph300.DEC, unit=(u.hourangle, u.deg) )
ra = coords.ra.radian
dec = coords.dec.radian

cosL300 = np.sin(dec)*np.sin(dec0) + np.cos(dec)*np.cos(dec0)*np.cos(ra-ra0)
eph300['aspect_angle'] = np.degrees(np.arccos(cosL300))

s300 = 2.5*np.log10( 1 - (1-R)*np.abs(cosL300) )



In [46]:
fig, ax = plt.subplots(figsize=fs.figsize(0.5),
                       gridspec_kw={
                           'bottom': 0.14, 
                           'top': 0.98, 
                           'left': 0.11, 
                           'right': 0.986
                       }
                      )





# Min/max of g
ax.plot( eph300.Phase, g300+np.min(s300), color='grey', zorder=-5 )
ax.plot( eph300.Phase, g300+np.max(s300), color='grey', zorder=-5 )

# --------------------------------------------------------------------------------
# Phase curve
x = eph300.Phase
y = g300 + s300
c = eph300.Date - jd_ref

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
norm = plt.Normalize(-50, 400)
lc = LineCollection(segments, cmap='Oranges', norm=norm)
lc.set_array(c)
lc.set_linewidth(2)
line = ax.add_collection(lc)

# Color bar
axinsr = inset_axes(
    ax,
    width="5%",  # width: 50% of parent_bbox width
    height="30%",  # height: 5%
    loc="lower left")
cbar = fig.colorbar(line, cax=axinsr, label='Days')
cbar.ax.set_yticks([0,100,200,300])
cbar.ax.tick_params(axis='y', length=10, direction='in')

# --------------------------------------------------------------------------------
# Illustrative dates
i_example = np.linspace(0,400,num=5, dtype=int)[1:-1]
markers = ['o','s','D']
for ii, i in enumerate(i_example):
    ax.scatter( eph300.loc[i,'Phase'], y[i], marker=markers[ii], facecolor='None', edgecolor='black', zorder=10 )
    ax.annotate( '{:4.0f}'.format(eph300.loc[i,'aspect_angle'])+'$^{\\textrm{o}}$',
                (eph300.loc[i,'Phase'], y[i]), 
                xytext=(2,2), 
               textcoords='offset points')

# --------------------------------------------------------------------------------
# Axes
ax.set_xlabel('Phase angle / deg.')
ax.set_ylabel('g($\gamma$) + s($\\alpha,\delta$)')
ax.set_ylim(0.95,-0.05)


# --------------------------------------------------------------------------------
# Orbit inserted
axorb = inset_axes(
    ax,
    width="30%",  # height: 5%
    height="50%",  # height: 5%
    loc="upper right")
axorb.set_aspect('equal')
axorb.plot( ephxyz.px, ephxyz.py, color='lightgrey' )
axorb.text( -3.5, -0.1, 'Orbit', color='lightgrey', fontsize='small')

x = xyz300.px
y = xyz300.py
c = xyz300.Date - jd_ref

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
norm = plt.Normalize(-50, 400)
lc = LineCollection(segments, cmap='Oranges', norm=norm)
lc.set_array(c)
lc.set_linewidth(2)
line = axorb.add_collection(lc)

# Earth
axorb.scatter(0,0, marker='o', s=4)

# Spin
spin_len = 1.4

x_sight = np.linspace(-5,0,num=10)
line_of_sight = x_sight * xyz300.loc[i_example[1],'py'] / xyz300.loc[i_example[1],'px']
axorb.plot( x_sight, line_of_sight )
for ii, i in enumerate(i_example):
    spin_pos = np.array([ xyz300.loc[i,'px'], xyz300.loc[i,'py']])
    spin_head = spin_pos + np.array( [ spin_len*np.cos(ra0), spin_len*np.sin(ra0) ])

    axorb.scatter( spin_pos[0], spin_pos[1], marker=markers[ii], facecolor='None', edgecolor='black', zorder=10 )

    axorb.annotate("", xy=spin_head, xytext=spin_pos,
                    arrowprops=dict(arrowstyle="->",color='black'))

axorb.text( spin_pos[0], spin_pos[1]-1.0, 'Spin axis', color='black',
            rotation=180+np.degrees( ra0 ), 
            ha='center', 
            fontsize='small')
    
# Line of sight
axorb.text( x_sight[4]+0.2, line_of_sight[4]-0.2, 'Line of sight', color=colors[0],
            rotation=180+np.degrees( np.arctan2( xyz300.loc[i_example[1],'py'],  xyz300.loc[i_example[1],'px'] ) ), 
            fontsize='small')
    
# Definition of Lambda = Aspect angle
i = i_example[1]
spin_pos = np.array([ xyz300.loc[i,'px'], xyz300.loc[i,'py']])

arc = Arc(spin_pos, spin_len, spin_len,
          theta1 = np.degrees(ra0), 
          theta2 = np.degrees( np.arctan2( xyz300.loc[i_example[1],'py'],  xyz300.loc[i_example[1],'px'] ) )
         )
axorb.add_patch(arc)
axorb.text( -4, -3.8, '$\\Lambda$')


# Axes
axorb.set_xlim(-4.5,.5)
axorb.set_ylim(-4.5,.5)
axorb.set_xticks(ticks=[])
axorb.set_yticks(ticks=[])
# axorb.set_xticklabels('')
# axorb.set_yticklabels('')


fig.savefig( os.path.join('..','gfx','article','fig_phase.png'), facecolor='white')
fig.savefig( os.path.join('..','gfx','article','fig_phase.pgf'), facecolor='white')
plt.close()

In [35]:
i_example

array([100, 200, 300])